In [1]:
import pandas as pd
from IPython.display import display
melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
# Сделаем преобразование столбцов к типу данных category
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца